In [9]:
!pip install --quiet openai

In [10]:
!pip install --upgrade --quiet pymupdf

In [4]:
import json

OPENAI_API_KEY = ''
with open('OpenAi.json', 'r') as file_to_read:
    json_data = json.load(file_to_read)
    OPENAI_API_KEY = json_data["OPENAI_API_KEY"]

In [5]:
import os
import openai

openai.api_key =  OPENAI_API_KEY

In [6]:
import fitz

In [7]:
doc = fitz.open('/content/AttentionAllYouNeed-2017.pdf') 

In [8]:
print("TotalPages of doc: ",len(doc))

TotalPages of doc:  15


In [ ]:
summary_list =[]
for page in doc:
  text = page.get_text("text")
  #print(text)
  prompt= text + "\n Tl;dr:"
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0.7,
    max_tokens=120,
    top_p=0.9,
    frequency_penalty=0.0,
    presence_penalty=1
  )
  summary_list.append(response["choices"][0]["text"])


In [ ]:
for i, summary_text in enumerate(summary_list):
  print(f"Page {i+1}: \n {summary_text}")

Page 1: 
 
 This paper proposes a new sequence transduction model based on attention mechanisms, which is superior in quality to existing models while requiring significantly less time to train. Experiments on two machine translation tasks show that the Transformer model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task and 41.8 BLEU on the WMT 2014 English-to-French translation task. The model also generalizes well to other tasks, such as English constituency parsing with both large and limited training data.
Page 2: 
  
 The Transformer is a model architecture that uses self-attention mechanisms to draw global dependencies between input and output, allowing for more parallelization and reaching a new state of the art in translation quality. Previous work such as Extended Neural GPU, ByteNet, and ConvS2S have used convolutional neural networks to compute hidden representations in parallel, but the Transformer takes this one step further by relying entirely on self-

In [ ]:
summary_text=' '.join(summary_list)
print(summary_text)


 This paper proposes a new sequence transduction model based on attention mechanisms, which is superior in quality to existing models while requiring significantly less time to train. Experiments on two machine translation tasks show that the Transformer model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task and 41.8 BLEU on the WMT 2014 English-to-French translation task. The model also generalizes well to other tasks, such as English constituency parsing with both large and limited training data.  
 The Transformer is a model architecture that uses self-attention mechanisms to draw global dependencies between input and output, allowing for more parallelization and reaching a new state of the art in translation quality. Previous work such as Extended Neural GPU, ByteNet, and ConvS2S have used convolutional neural networks to compute hidden representations in parallel, but the Transformer takes this one step further by relying entirely on self-attention. 
The Tran

## Question answer with text embedding in summarized paper.

In [ ]:
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity

In [ ]:
df = pd.DataFrame(summary_list)
print(df.shape)
df.head()


(15, 1)


,0
0,\n This paper proposes a new sequence transduc...
1,\n The Transformer is a model architecture th...
2,\nThe Transformer model architecture consists ...
3,Scaled dot-product attention is a type of att...
4,\n Multi-head attention allows the model to at...


In [ ]:
embedding_model = "text-embedding-ada-002"
embeddings = df[0].astype(str).apply([lambda x: get_embedding(x, engine=embedding_model)])
df["embeddings"] = embeddings

In [ ]:
df.shape

(15, 2)

In [ ]:
def search_reviews(df, query):
    query_embedding = get_embedding(
        query,
        engine="text-embedding-ada-002"
    )
    df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
    )
    
    return results

In [ ]:
results = search_reviews(df, "what is this paper about?")
results.iloc[0][0]


"\n \nThis paper presents a review of recent advances in neural machine translation, summarization, parsing and language modeling. It covers models such as encoder-decoders, attention mechanisms, tree-based models, deep reinforced models, output embeddings, subword units, sparsely-gated mixture-of-experts layers, and end-to-end memory networks. It also discusses popular approaches such as dropout, Google's Neural Machine Translation system, Fast and Accurate Shift-Reduce Constituent Parsing, and Grammar as a Foreign Language."

In [ ]:
results = search_reviews(df, "explain the training procedure")
results.iloc[0][0]

' We describe our training regime for the Transformer model, including data and batching, hardware and schedule, optimizer, and regularization. We use Adam optimizer with warmup steps and residual dropout.'

In [ ]:
results = search_reviews(df, "Which one is better, transformer or RNN")
results.iloc[0][0]

'\n \nThis paper presents the Transformer, a sequence transduction model based entirely on attention. It is shown to be able to outperform other architectures based on recurrent or convolutional layers in translation tasks. The Transformer is trained faster and achieves state-of-the-art results for English-to-German and English-to-French translation tasks. We plan to extend the Transformer to other tasks and investigate local, restricted attention mechanisms for large inputs and outputs.'

## Rerun with summarized text give even more summarized version.

In [ ]:
prompt= summary_text + "\n Tl;dr:"
response = openai.Completion.create(
model="text-davinci-003",
prompt=prompt,
temperature=0.7,
max_tokens=400,
top_p=0.9,
frequency_penalty=0.0,
presence_penalty=1
)


In [ ]:
short_summary = response["choices"][0]["text"]
print("The short summary is: \n", short_summary)

The short summary is: 
  This paper reviews recent advances in sequence transduction models based on attention mechanisms, such as the Transformer model. The Transformer is shown to be superior in quality to existing models while requiring significantly less time to train. Experiments on machine translation tasks and English constituency parsing show that the Transformer model achieves state-of-the-art results. Self-attention layers are used in the encoder and decoder stack and allow for more parallelization than recurrent or convolutional layers.


In [ ]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " This paper presents a new architecture, the Transformer, which relies on attention mechanisms instead of recurrence and is more parallelizable. Experiments on two machine translation tasks show that the model achieves superior results, improving over the existing best results by 2 BLEU points. It contains an encoder and decoder stack with multi-head self-attention mechanisms and position-wise fully connected feed-forward networks. Attention heads in the encoder self-attention layer have learned to perform different tasks related to the structure of the sentence."
    }
  ],
  "created": 1670219783,
  "id": "cmpl-6Jykhkghj0VSQG0tokyjPRpg6w9OA",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 108,
    "prompt_tokens": 1262,
    "total_tokens": 1370
  }
}


In [ ]:
summary_list =[]
for page in doc:
  text = page.get_text("text")
  #print(text)
  prompt= "Summarize this for a second-grade student: " +text 
  response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0.7,
  max_tokens=120,
  top_p=0.9,
  frequency_penalty=0.0,
  presence_penalty=1
  )
  summary_list.append(response["choices"][0]["text"])


In [ ]:
summary_text= ' '.join(summary_list)
print(summary_text)


Scientists have created a new type of network architecture called the Transformer that uses attention mechanisms instead of recurrence and convolutions to help machines understand and translate language. Experiments have shown that this model is faster and more accurate than other models that have been used before. It has helped machines to do things like translate English into German or French and understand how words fit together in a sentence. A Transformer is a type of model used for language modeling and machine translation. It uses attention mechanisms to draw global dependencies between input and output, which allows for more parallelization and faster training times. For a second-grade student:
The Transformer is a model that helps machines understand and generate language. It has two parts - an encoder which takes in the language and a decoder which produces the output language. It also uses something called attention, which is like a way for the machine to remember important

In [ ]:
prompt= "Summarize this for a second-grade student: "+summary_text 
response = openai.Completion.create(
model="text-davinci-003",
prompt=prompt,
temperature=0.7,
max_tokens=400,
top_p=0.9,
frequency_penalty=0.0,
presence_penalty=1
)
print(response["choices"][0]["text"])

 Scientists have been looking for ways to make computers understand language better. They have created a model called the Transformer that uses attention mechanisms and is able to translate quickly and accurately. This model has been tested on two machine translation tasks and was found to be more accurate and faster than existing methods. It also works well for English constituency parsing.
